In [1]:
import os
import pyspark.sql.functions as F

from pyspark.sql import SparkSession



# Caminho para o arquivo de credenciais
credentials_path = "/home/jovyan/work/desafio-stone-439013-cd4f41db04c5.json"

# Configurar a variável de ambiente para as credenciais do Google Cloud
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

# Crie a sessão do Spark com o conector BigQuery
spark = (
    SparkSession.builder
    .appName('BigQuery Crypto Ethereum')
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.30.0,org.postgresql:postgresql:42.2.23') \
    .config("spark.sql.execution.arrow.enabled", "true")
    .getOrCreate()
)
spark

In [8]:
# Lendo dados do BigQuery
project_id = "desafio-stone-439013"
bigquery_table = "bigquery-public-data.crypto_ethereum.tokens"
df = (
    spark.read
    .format("bigquery") 
    .option("project", project_id) 
    .option("table", bigquery_table)
    .option("filter", "DATE(block_timestamp) >= '2024-01-01'") \
    .load()
)

# Exibir os dados
df.show(5)

+--------------------+---------+------------+--------+--------------------+-------------------+------------+--------------------+
|             address|   symbol|        name|decimals|        total_supply|    block_timestamp|block_number|          block_hash|
+--------------------+---------+------------+--------+--------------------+-------------------+------------+--------------------+
|0x8c9878df204f813...|      ETH|      ETH...|      18|10000000000000000...|2024-01-07 03:48:11|    18952658|0x1a30b646b63e1b4...|
|0x2350f30623bdfd9...| MRHOODIE|NodeMrHoodie|    NULL|                  70|2024-01-09 07:07:23|    18967818|0x24854672de99f3f...|
|0x9dfceb2c72c395f...|     SHIB|   Shiba Inu|      18|10000000000000000...|2024-01-12 23:07:35|    18993992|0xacda38289e815eb...|
|0x948ba8fb1ec990d...|BabyTroll|   BabyTroll|      18|21000000000000000...|2024-01-15 13:16:11|    19012519|0xb722228a9836c78...|
|0x90e24c08f767b80...|    ORDII|       ORDII|       0|            21000000|2024-01-05 13:0

In [9]:
# Conexão com o PostgreSQL
postgres_url = "jdbc:postgresql://10.109.6.155:5432/crypto_ethereum"
postgres_properties = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}

# Escreve os dados no PostgreSQL
df.write.jdbc(url=postgres_url, table="public.crypto_tokens", mode="append", properties=postgres_properties)